# Menganalisis Risiko Gagal Bayar Peminjam

# Konten <a id='back'></a>

* [Pendahuluan](#intro)
    * [1.1 Tujuan](#tujuan)
    * [1.2 Tahapan](#tahap)
* [Tahap 1: Membuka *file* data dan memahami informasi umumnya](#data_review)
* [Tahap 2: Eksplorasi data](#data_exploring)
* [Tahap 3: Transformasi data](#transforming_data)
    * [4.1 Memperbaiki nilai unik](#unique)
    * [4.2 Menangani duplikat](#duplicate)
* [Tahap 4: Bekerja dengan nilai yang hilang](#missing_values)
    * [5.1 Memperbaiki nilai yang hilang di total_income](#missing_total_income)
    * [5.2 Memperbaiki nilai yang hilang di days_employed](#missing_days_employed)
* [Tahap 5: Pengkategorian Data](#categorize)
* [Tahap 6: Memeriksa Hipotesis](#hypotheses)
    * [Hipotesis 1](#hypotheses_1)
    * [Hipotesis 2](#hypotheses_2)
    * [Hipotesis 3](#hypotheses_3)
    * [Hipotesis 4](#hypotheses_4)
* [Kesimpulan](#end)


## Pendahuluan <a id='intro'></a>

Data berikut merupakan data mengenai nasabah bank yang akan digunakan divisi di suatu bank untuk dijadikan bahan pertimbangan pada saat membuat **penilaian kredit** sebagai evaluasi kemampuan calon nasabah untuk melunasi pinjaman mereka. Melalui beberapa data mengenai kelayakan kredit nasabah yang telah disediakan pihak bank, kita akan mencari tahu pengaruh status perkawinan seorang nasabah dan jumlah anak yang dimilikinya terhadap probabilitas gagal bayar dalam pelunasan pinjaman.

### Tujuan: <a id='tujuan'></a>

Kita akan menguji 4 hipotesis:
1. Apakah terdapat korelasi antara memiliki anak dengan melakukan pelunasan tepat waktu?
2. Apakah terdapat korelasi antara status keluarga dengan pelunasan tepat waktu?
3. Apakah terdapat korelasi antara tingkat pendapatan dengan membayar kembali tepat waktu?
4. Bagaimana tujuan kredit memengaruhi tingkat gagal bayar?

### Tahapan: <a id='tahap'></a>

Tidak ada informasi tentang kualitas data, oleh karena itu kita perlu memeriksanya terlebih dahulu sebelum menguji hipotesis.

Pertama,evaluasi kualitas datanya dan melihat apakah masalahnya signifikan. Kemudian kita akan mencoba memperhitungkan masalah yang lebih serius selama eksplorasi data berlangsung.
 
Proyek ini akan terdiri dari 6 tahap:
 1. Membuka *file* data dan memahami informasi umumnya
 2. Eksplorasi data
 3. Transformasi data
 4. Bekerja dengan nilai yang hilang
 5. Pengkategorian Data
 6. Memeriksa Hipotesis

## Tahap 1: Membuka *file* data dan memahami informasi umumnya. <a id='data_review'></a>

In [1]:
# Memuat semua *library*
import pandas as pd
import numpy as np

In [2]:
# Memuat datanya
try:
    credit_scoring = pd.read_csv('credit_scoring_eng.csv')
except:
    credit_scoring = pd.read_csv('/datasets/credit_scoring_eng.csv')

In [3]:
credit_scoring.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


## Tahap 2: Eksplorasi data <a id='data_exploring'></a>

**Deskripsi Data**
- `children` - jumlah anak dalam keluarga
- `days_employed` - pengalaman kerja nasabah dalam hari
- `dob_years` - usia nasabah dalam tahun
- `education` - tingkat pendidikan nasabah
- `education_id` - pengidentifikasi untuk tingkat pendidikan nasabah
- `family_status` - pengidentifikasi untuk status perkawinan nasabah
- `family_status_id` - tanda pengenal status perkawinan
- `gender` - jenis kelamin nasabah
- `income_type` - jenis pekerjaan
- `debt` - apakah nasabah memiliki hutang pembayaran pinjaman
- `total_income` - pendapatan bulanan
- `purpose` - tujuan mendapatkan pinjaman

Mari lihat ukuran keseluruhan datasetnya

In [4]:
# Mari kita lihat berapa banyak baris dan kolom yang dimiliki oleh dataset kita

credit_scoring.shape

(21525, 12)

Keseluruhan *DataFrame* memiliki 21525 baris dan 12 kolom. 

In [5]:
# Mari tampilkan 15 baris pertama

credit_scoring.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


Pada kolom `education` dan `purpose` terdapat duplikat implisit yang dikarenakan cara penulisan yang berbeda. Selain itu kolom `days_employed` memiliki nilai yang cukup janggal, dimana nilai merupakan jumlah hari yang mana tidak mungkin nilainya negatif. 

Selanjutnya mari kita lihat informasi datanya.

In [6]:
# Mendapatkan informasi data
credit_scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [7]:
credit_scoring.describe(include='all')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
count,21525.000000,19351.000000,21525.000000,21525,21525.000000,21525,21525.000000,21525,21525,21525.000000,19351.000000,21525
unique,NaN,NaN,NaN,15,NaN,5,NaN,3,8,NaN,NaN,38
top,NaN,NaN,NaN,secondary education,NaN,married,NaN,F,employee,NaN,NaN,wedding ceremony
freq,NaN,NaN,NaN,13750,NaN,12380,NaN,14236,11119,NaN,NaN,797
mean,0.538908,63046.497661,43.293380,NaN,0.817236,NaN,0.972544,NaN,NaN,0.080883,26787.568355,NaN
std,1.381587,140827.311974,12.574584,NaN,0.548138,NaN,1.420324,NaN,NaN,0.272661,16475.450632,NaN
min,-1.000000,-18388.949901,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN,0.000000,3306.762000,NaN
25%,0.000000,-2747.423625,33.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,16488.504500,NaN
50%,0.000000,-1203.369529,42.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,23202.870000,NaN
75%,1.000000,-291.095954,53.000000,NaN,1.000000,NaN,1.000000,NaN,NaN,0.000000,32549.611000,NaN


Pada kolom `days_employed` dan `total_income` hanya memiliki 19351 dari total 21525 data yang artinya terdapat nilai yang hilang pada kolom tersebut. Uniknya keduanya memiliki jumlah nilai yang sama sehingga kemungkinan keduanya memiliki keterkaintan.

Kolom `children` memiliki nilai yang mencurigakan dimana nilai minimumnya adalah -1 dan niai maksimumnya 20, angka yang cukup aneh untuk menunjukan jumlah anak yang dimiliki seorang nasabah.

Mari kita lihat datanya.

In [8]:
# Mari kita lihat tabel yang difilter dengan nilai yang hilang di kolom pertama yang mengandung data yang hilang

credit_scoring.loc[credit_scoring['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


Nilai yang hilang pada kolom `days_employed` dan `total_income` memiliki pola yang simetris dimana ketika data di kolom `days_employed` hilang, begitu pula data pada kolom `total_income`. Mari lakukan penyelidikan lebih lanjut dan menghitung nilai yang hilang di semua baris untuk memastikan bahwa data yang hilang memiliki ukuran yang sama.

In [9]:
# Mari kita terapkan beberapa kondisi untuk memfilter data dan melihat jumlah baris dalam tabel yang telah difilter.

credit_scoring_clear = credit_scoring.dropna()
credit_scoring_clear

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


In [10]:
# Menghitung jumlah nilai yang hilang

len(credit_scoring) - len(credit_scoring_clear)

2174

**Kesimpulan sementara**

Setelah memfilter data yang hilang dari kolom `days_employed` ternyata nilai yang hilang tampak simetris dengan nilai yang hilang pada kolom `total_income`, selain itu jumlah nilai yang hilang di kedua kolom ternyata sama yaitu 2174 baris. Artinya, kedua kolom tersebut saling berkaitan satu sama lain. 

Selanjutnya mari hitung presentase nilai yang hilang dengan keseluruhan data untuk melihat seberapa besar pengaruh data yang hilang kepada keseluruhan data.

In [11]:
# Presentase nilai yang hilang terhadap keseluruhan dataset

credit_scoring.isna().sum() / len(credit_scoring) * 100

children             0.000000
days_employed       10.099884
dob_years            0.000000
education            0.000000
education_id         0.000000
family_status        0.000000
family_status_id     0.000000
gender               0.000000
income_type          0.000000
debt                 0.000000
total_income        10.099884
purpose              0.000000
dtype: float64

Presentase nilai yang hilang cukup besar yaitu sekitar 10.1%, dengan demikian nilai yang hilang harus diisi. Untuk melakukannya, pertama-tama kita harus mempertimbangkan apakah data yang hilang bisa jadi disebabkan oleh karakteristik nasabah tertentu, seperti jenis pekerjaan atau yang lainnya. Anda harus memutuskan karakteristik mana yang menurut Anda mungkin merupakan penyebabnya. Kedua, kita harus memeriksa apakah ada ketergantungan nilai yang hilang pada nilai indikator lain dengan kolom-kolom yang mengidentifikasikan karakteristik tertentu nasabah.

In [12]:
# Mari kita periksa nasabah yang tidak memiliki data tentang karakteristik yang teridentifikasi dan kolom dengan nilai yang hilang

sorted(credit_scoring.loc[(credit_scoring['total_income'].isna()), 'income_type'].unique())

['business', 'civil servant', 'employee', 'entrepreneur', 'retiree']

In [13]:
# Memeriksa distribusinya

sorted(credit_scoring['income_type'].unique())

['business',
 'civil servant',
 'employee',
 'entrepreneur',
 'paternity / maternity leave',
 'retiree',
 'student',
 'unemployed']

Nasabah dengan nilai yang hilang merupakan nasabah yang sumber penghasilannya berasal dari `'business'`, `'civil servant'`, `'employee'`, `'entrepreneur'`, dan `'retiree'`.

**Kemungkinan penyebab hilangnya nilai dalam data**

Nilai yang hilang kemungkinan disebabkan oleh kesalan pada sistem dimana data pada kolom `days_imployed` yang hilang dan menyebabkan nilai pada kolom `total_income` ikut hilang.

**Memeriksa apakah nilai yang hilang bersifat acak**

In [14]:
credit_scoring.loc[(credit_scoring['income_type']=='business') & (credit_scoring['total_income'].isna())]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
65,0,NaN,21,secondary education,1,unmarried,4,M,business,0,NaN,transactions with commercial real estate
94,1,NaN,34,bachelor's degree,0,civil partnership,1,F,business,0,NaN,having a wedding
121,0,NaN,29,bachelor's degree,0,married,0,F,business,0,NaN,car
135,0,NaN,27,secondary education,1,married,0,M,business,0,NaN,housing
174,0,NaN,55,bachelor's degree,0,widow / widower,2,F,business,0,NaN,to own a car
...,...,...,...,...,...,...,...,...,...,...,...,...
21390,20,NaN,53,secondary education,1,married,0,M,business,0,NaN,buy residential real estate
21391,0,NaN,52,secondary education,1,married,0,F,business,0,NaN,purchase of the house for my family
21407,1,NaN,36,secondary education,1,married,0,F,business,0,NaN,building a real estate
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car


In [15]:
credit_scoring.loc[(credit_scoring['income_type']=='business') & ~(credit_scoring['total_income'].isna())]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
10,2,-4171.483647,36,bachelor's degree,0,married,0,M,business,0,18230.959,buy real estate
14,0,-1844.956182,56,bachelor's degree,0,civil partnership,1,F,business,1,26420.466,buy residential real estate
33,0,-1548.637544,48,secondary education,1,married,0,F,business,0,25159.326,purchase of the house
...,...,...,...,...,...,...,...,...,...,...,...,...
21512,0,-165.377752,26,bachelor's degree,0,unmarried,4,M,business,0,23568.233,to get a supplementary education
21514,0,-280.469996,27,some college,2,unmarried,4,M,business,0,56958.145,building a property
21516,0,-914.391429,42,bachelor's degree,0,married,0,F,business,0,51649.244,purchase of my own house
21517,0,-404.679034,42,bachelor's degree,0,civil partnership,1,F,business,0,28489.529,buying my own car


Berdasarkan data yang sudah difilter berdasarkan salah satu value dari kolom `income_type` yaitu `'business'`, kita menemukan bahwa nilai yang hilang bersifat acak karena hanya sebagian kecil data saja yang hilang dalam satu unique value tersebut.

Selanjutnya mari kita periksa distribusi keseluruhan dataset.

In [16]:
# Memeriksa distribusi kolom `children` dan membandingkannya dengan DataFrame yang telah difilter

print(credit_scoring['children'].nunique())
print(credit_scoring['children'].unique())
print()
print(credit_scoring_clear['children'].nunique())
print(credit_scoring_clear['children'].unique())

8
[ 1  0  3  2 -1  4 20  5]

8
[ 1  0  3  2 -1  4 20  5]


In [17]:
# Memeriksa distribusi kolom `children` dan membandingkannya dengan DataFrame yang telah difilter

print(credit_scoring['children'].value_counts())
print()
print(credit_scoring_clear['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

 0     12710
 1      4343
 2      1851
 3       294
 20       67
-1        44
 4        34
 5         8
Name: children, dtype: int64


In [18]:
pd.DataFrame({"percent": credit_scoring_clear.children.value_counts(normalize=True).mul(100).round(2).astype(str) + '%',
              "counter" : credit_scoring_clear.children.value_counts()
            }) # Chamdani code example

,percent,counter
0,65.68%,12710
1,22.44%,4343
2,9.57%,1851
3,1.52%,294
20,0.35%,67
-1,0.23%,44
4,0.18%,34
5,0.04%,8


In [19]:
# Memeriksa distribusi kolom `education` dan membandingkannya dengan DataFrame yang telah difilter

print(credit_scoring['education'].nunique())
print(credit_scoring['education'].unique())
print()
print(credit_scoring_clear['education'].nunique())
print(credit_scoring_clear['education'].unique())

15
["bachelor's degree" 'secondary education' 'Secondary Education'
 'SECONDARY EDUCATION' "BACHELOR'S DEGREE" 'some college'
 'primary education' "Bachelor's Degree" 'SOME COLLEGE' 'Some College'
 'PRIMARY EDUCATION' 'Primary Education' 'Graduate Degree'
 'GRADUATE DEGREE' 'graduate degree']

15
["bachelor's degree" 'secondary education' 'Secondary Education'
 'SECONDARY EDUCATION' "BACHELOR'S DEGREE" 'some college'
 'primary education' "Bachelor's Degree" 'SOME COLLEGE' 'Some College'
 'PRIMARY EDUCATION' 'Primary Education' 'Graduate Degree'
 'GRADUATE DEGREE' 'graduate degree']


In [20]:
# Memeriksa distribusi kolom `education` dan membandingkannya dengan DataFrame yang telah difilter

print(credit_scoring['education'].value_counts())
print()
print(credit_scoring_clear['education'].value_counts())

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

secondary education    12342
bachelor's degree       4222
SECONDARY EDUCATION      705
Secondary Education      646
some college             613
BACHELOR'S DEGREE        251
Bachelor's Degree        243
primary education        231
Some College              40
SOME COLLEGE              22
PRIMARY EDUCATION         16
Primary Education         14
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64


In [21]:
# Memeriksa distribusi kolom `education_id` dan membandingkannya dengan DataFrame yang telah difilter

print(credit_scoring['education_id'].nunique())
print(credit_scoring['education_id'].unique())
print()
print(credit_scoring_clear['education_id'].nunique())
print(credit_scoring_clear['education_id'].unique())

5
[0 1 2 3 4]

5
[0 1 2 3 4]


In [22]:
# Memeriksa distribusi kolom `education_id` dan membandingkannya dengan DataFrame yang telah difilter

print(credit_scoring['education_id'].value_counts())
print()
print(credit_scoring_clear['education_id'].value_counts())

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

1    13693
0     4716
2      675
3      261
4        6
Name: education_id, dtype: int64


In [23]:
# Memeriksa distribusi kolom `family_status` dan membandingkannya dengan DataFrame yang telah difilter

print(credit_scoring['family_status'].nunique())
print(credit_scoring['family_status'].unique())
print()
print(credit_scoring_clear['family_status'].nunique())
print(credit_scoring_clear['family_status'].unique())

5
['married' 'civil partnership' 'widow / widower' 'divorced' 'unmarried']

5
['married' 'civil partnership' 'widow / widower' 'divorced' 'unmarried']


In [24]:
# Memeriksa distribusi kolom `family_status` dan membandingkannya dengan DataFrame yang telah difilter

print(credit_scoring['family_status'].value_counts())
print()
print(credit_scoring_clear['family_status'].value_counts())

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

married              11143
civil partnership     3735
unmarried             2525
divorced              1083
widow / widower        865
Name: family_status, dtype: int64


In [25]:
# Memeriksa distribusi kolom `family_status_id` dan membandingkannya dengan DataFrame yang telah difilter

print(credit_scoring['family_status_id'].nunique())
print(credit_scoring['family_status_id'].unique())
print()
print(credit_scoring_clear['family_status_id'].nunique())
print(credit_scoring_clear['family_status_id'].unique())

5
[0 1 2 3 4]

5
[0 1 2 3 4]


In [26]:
# Memeriksa distribusi kolom `family_status_id` dan membandingkannya dengan DataFrame yang telah difilter

print(credit_scoring['family_status_id'].value_counts())
print()
print(credit_scoring_clear['family_status_id'].value_counts())

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

0    11143
1     3735
4     2525
3     1083
2      865
Name: family_status_id, dtype: int64


In [27]:
# Memeriksa distribusi kolom `purpose` dan membandingkannya dengan DataFrame yang telah difilter

print(credit_scoring['purpose'].nunique())
print(credit_scoring['purpose'].unique())
print()
print(credit_scoring_clear['purpose'].nunique())
print(credit_scoring_clear['purpose'].unique())

38
['purchase of the house' 'car purchase' 'supplementary education'
 'to have a wedding' 'housing transactions' 'education' 'having a wedding'
 'purchase of the house for my family' 'buy real estate'
 'buy commercial real estate' 'buy residential real estate'
 'construction of own property' 'property' 'building a property'
 'buying a second-hand car' 'buying my own car'
 'transactions with commercial real estate' 'building a real estate'
 'housing' 'transactions with my real estate' 'cars' 'to become educated'
 'second-hand car purchase' 'getting an education' 'car'
 'wedding ceremony' 'to get a supplementary education'
 'purchase of my own house' 'real estate transactions'
 'getting higher education' 'to own a car' 'purchase of a car'
 'profile education' 'university education'
 'buying property for renting out' 'to buy a car' 'housing renovation'
 'going to university']

38
['purchase of the house' 'car purchase' 'supplementary education'
 'to have a wedding' 'housing transactions' 

In [28]:
# Memeriksa distribusi kolom `purpose` dan membandingkannya dengan DataFrame yang telah difilter

print(credit_scoring['purpose'].value_counts())
print()
print(credit_scoring_clear['purpose'].value_counts())

wedding ceremony                            797
having a wedding                            777
to have a wedding                           774
real estate transactions                    676
buy commercial real estate                  664
buying property for renting out             653
housing transactions                        653
transactions with commercial real estate    651
purchase of the house                       647
housing                                     647
purchase of the house for my family         641
construction of own property                635
property                                    634
transactions with my real estate            630
building a real estate                      626
buy real estate                             624
building a property                         620
purchase of my own house                    620
housing renovation                          612
buy residential real estate                 607
buying my own car                       

**Kesimpulan sementara**

Distribusi dalam *dataset* yang asli mirip dengan distribusi tabel yang telah difilter, mungkin kita masih harus memastikan sekali lagi apakah ada pola tertentu yang menyebabkan hilangnya nilai-nilai tersebut tidak terjadi secara acak.

In [29]:
# Memeriksa penyebab dan pola lain yang dapat mengakibatkan nilai yang hilang

print(credit_scoring.loc[~(credit_scoring['total_income'].isna()), 'income_type'].nunique())
print(credit_scoring.loc[~(credit_scoring['total_income'].isna()), 'income_type'].unique())

8
['employee' 'retiree' 'business' 'civil servant' 'unemployed' 'student'
 'entrepreneur' 'paternity / maternity leave']


In [30]:
# Memeriksa penyebab dan pola lain yang dapat mengakibatkan nilai yang hilang

print(credit_scoring.loc[~(credit_scoring['total_income'].isna()), 'income_type'].value_counts())

employee                       10014
business                        4577
retiree                         3443
civil servant                   1312
unemployed                         2
student                            1
entrepreneur                       1
paternity / maternity leave        1
Name: income_type, dtype: int64


In [31]:
print(credit_scoring['income_type'].nunique())
print(credit_scoring['income_type'].unique())

8
['employee' 'retiree' 'business' 'civil servant' 'unemployed'
 'entrepreneur' 'student' 'paternity / maternity leave']


In [32]:
print(credit_scoring['income_type'].value_counts())

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64


**Kesimpulan**

Kita telah mengetahui bahwa nilai yang hilang berada pada kolom `days_employed` dan `total_income`, dimana data yang hilang memiliki pola yang simetris karena ketika data di kolom `days_employed` hilang, begitu pula data pada kolom `total_income`. Hal ini bisa terjadi kemungkinan karena adanya kesalahan dalam sistem dan bersifat acak. Kitapun telah mengetahui bahwa nasabah dengan nilai yang hilang merupakan nasabah yang sumber penghasilannya berasal dari `'business'`, `'civil servant'`, `'employee'`, `'entrepreneur'`, dan `'retiree'`.

Berdasarkan *unique value* yang kita dapatkan, nilai yang hilang merupakan suatu kebetulan dikarenakan jumlah unique value yang sama antara data yang sudah difilter dan data yang tidak difilter.

Setelah dihitung, presentase nilai yang hilang memiliki jumlah yang cukup signifikan yaitu sebanyak 10.1%, oleh karenanya maka kita perlu mengisi nilai yang hilang. Kita bisa menggisi nilai yang hilang dengan nilai rata-rata atau median dari masing-masing kolom yang memiliki nilai yang hilang.

Sebelum mengisi nilai yang hilang, perlu dilakukan **Transfotmasi Data** terlebih dahulu. Kita mungkin perlu mengatasi berbagai jenis masalah seperti duplikat, pencatatan yang berbeda, sumber data yang salah, dan barulah kita menangani nilai yang hilang.

## Tahap 3: Transformasi data <a id='transforming_data'></a>

### Memperbaiki nilai unik <a id='unique'></a>

Kita akan memeriksa nilai unik pada setiap kolom untuk melihat masalah apa yang mungkin dimiliki oleh masing-masing kolom. Kita akan memulainya dengan kolom `education`.

**Kolom `education`**

In [33]:
# Mari kita lihat semua nilai di kolom pendidikan untuk memeriksa ejaan apa yang perlu diperbaiki
sorted(credit_scoring['education'].unique())

["BACHELOR'S DEGREE",
 "Bachelor's Degree",
 'GRADUATE DEGREE',
 'Graduate Degree',
 'PRIMARY EDUCATION',
 'Primary Education',
 'SECONDARY EDUCATION',
 'SOME COLLEGE',
 'Secondary Education',
 'Some College',
 "bachelor's degree",
 'graduate degree',
 'primary education',
 'secondary education',
 'some college']

Kolom `education` memiliki nilai duplikat namun dengan cara penulisan yang berbeda sehingga dianggap nilai unik. Mari perbaiki dengan cara mengubah penulisannya menjadi huruf kecil.

In [34]:
# Memperbaiki pencatatan jika diperlukan
credit_scoring['education'].str.lower().unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

In [35]:
# Periksa semua nilai di kolom untuk memastikan bahwa kita telah memperbaikinya dengan tepat

credit_scoring['education'] = credit_scoring['education'].str.lower()
sorted(credit_scoring['education'].unique())

["bachelor's degree",
 'graduate degree',
 'primary education',
 'secondary education',
 'some college']

Nilai pada kolom `education` telah diperbaiki.

**Kolom `children`**

Selanjutnya mari periksa data kolom `children`:

In [36]:
# Mari kita lihat distribusi nilai pada kolom `children`
sorted(credit_scoring['children'].unique())

[-1, 0, 1, 2, 3, 4, 5, 20]

Pada kolom `children` terdapat nilai yang mencurigakan dimana ada yang memiliki nilai -1 dan 20. Hal ini cukup mencurigakan karena tidak mungkin seseorang memiliki anak dengan jumlah negatif dan juga saat ini jarang sekali kemungkinan orang yang memiliki 20 anak. Ini bisa saja merupakan kesalahan input maupun kesalahan ketik dimana mungkin nilai yang dimaksud adalah 1 atau 2.

In [37]:
credit_scoring.loc[(credit_scoring['children']==20)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,secondary education,1,married,0,M,business,0,23253.578,purchase of the house
720,20,-855.595512,44,secondary education,1,married,0,F,business,0,18079.798,buy real estate
1074,20,-3310.411598,56,secondary education,1,married,0,F,employee,1,36722.966,getting an education
2510,20,-2714.161249,59,bachelor's degree,0,widow / widower,2,F,employee,0,42315.974,transactions with commercial real estate
2941,20,-2161.591519,0,secondary education,1,married,0,F,employee,0,31958.391,to buy a car
...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,-1240.257910,40,secondary education,1,married,0,F,employee,1,21363.842,to own a car
21325,20,-601.174883,37,secondary education,1,married,0,F,business,0,16477.771,profile education
21390,20,NaN,53,secondary education,1,married,0,M,business,0,NaN,buy residential real estate
21404,20,-494.788448,52,secondary education,1,married,0,M,business,0,25060.749,transactions with my real estate


In [38]:
credit_scoring.loc[(credit_scoring['children']==-1)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,-4417.703588,46,secondary education,1,civil partnership,1,F,employee,0,16450.615,profile education
705,-1,-902.084528,50,secondary education,1,married,0,F,civil servant,0,22061.264,car purchase
742,-1,-3174.456205,57,secondary education,1,married,0,F,employee,0,10282.887,supplementary education
800,-1,349987.852217,54,secondary education,1,unmarried,4,F,retiree,0,13806.996,supplementary education
941,-1,NaN,57,secondary education,1,married,0,F,retiree,0,NaN,buying my own car
1363,-1,-1195.264956,55,secondary education,1,married,0,F,business,0,11128.112,profile education
1929,-1,-1461.303336,38,secondary education,1,unmarried,4,M,employee,0,17459.451,purchase of the house
2073,-1,-2539.761232,42,secondary education,1,divorced,3,F,business,0,26022.177,purchase of the house
3814,-1,-3045.290443,26,secondary education,1,civil partnership,1,F,civil servant,0,21102.846,having a wedding
4201,-1,-901.101738,41,secondary education,1,married,0,F,civil servant,0,36220.123,transactions with my real estate


Uniknya ada cukup banyak data yang memiliki nilai -1 dan 20 pada kolom `children`.

In [39]:
# Memperbaiki nilai pada kolom `children`
credit_scoring['children'] = credit_scoring['children'].replace([-1,20], [1,2])

In [40]:
# Memeriksa kembali kolom `children` untuk memastikan bahwa semuanya telah diperbaiki

sorted(credit_scoring['children'].unique())

[0, 1, 2, 3, 4, 5]

Kolom `children` telah diperbaiki.

**Kolom `days_employed`**

Mari beralih ke kolom `days_employed`, kolom ini memiliki nilai yang mencurigakan yaitu nilai negatif. Berikut data dengan nilai negatif :

In [41]:
# Menemukan data yang bermasalah di `days_employed`

credit_scoring.loc[(credit_scoring['days_employed'] < 0)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
...,...,...,...,...,...,...,...,...,...,...,...,...
21519,1,-2351.431934,37,graduate degree,4,divorced,3,M,employee,0,18551.846,buy commercial real estate
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


Menghitung presentase data yang bermasalah untuk memastikan seberapa besar datanya.

In [42]:
# Menghitung presentase data yang bermasalah

len(credit_scoring.loc[(credit_scoring['days_employed'] < 0)]) / len(credit_scoring) * 100

73.89547038327527

Presentase data yang bermasalah ternyata cukup tinggi yaitu sekitar 73.90%, hal ini mungkin disebabkan oleh beberapa masalah teknis sehingga nilai yang seharusnya positif berubah menjadi negatif. Nilai yang bermasalah tersebut bisa diatasi dengan mengubah data yang bernilai negatif menjadi positif.

In [43]:
# Mengatasi data yang bermasalah

credit_scoring['days_employed'] = credit_scoring['days_employed'].abs()

In [44]:
# Memastikan bahwa masalahnya telah diperbaiki
credit_scoring.loc[(credit_scoring['days_employed'] < 0)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Nilai yang bermasalah pada kolom `days_employed` berupa nilai negatif telah diperbaiki, namun kita masih harus memastikan apakah masih ada data yang bermasalah.

In [45]:
# Memastikan kembali apakah masih ada data yang bermasalah

credit_scoring['days_employed'] / 365

0         23.116912
1         11.026860
2         15.406637
3         11.300677
4        932.235814
            ...    
21520     12.409087
21521    942.294258
21522      5.789991
21523      8.527347
21524      5.437007
Name: days_employed, Length: 21525, dtype: float64

Ternyata masih ada data yang mencurigakan dimana jumlah tahun bekerja beberapa nasabah menyentuhh angka sembilan ratusan. Hal ini bisa jadi merupakan kesalahan data mengingat rekor manusia tertua di dunia saat ini yang di pegang oleh Jeanne Calment hanya 122 tahun dan 164 hari.

Kita akan menghapus data dengan tahun bekerja yang tidak masuk akal karena akan mempengaruhi nilai rata-rata dan nilai median yang nantinya akan digunakan untuk mengisi nilai yang hilang. Sebelumnya kita harus menemukan rata-rata umur produktif seseorang. Menurut [DataIndonesia.id](https://dataindonesia.id/undefined/detail/mayoritas-penduduk-indonesia-merupakan-umur-produktif) rata-rata usia produktif seseorang adalah 15-64 tahun, atau sekitar 49 tahun.

**Mari kita buat kolom `years_employed`**

In [46]:
# Membuat kolom `years_employed`

credit_scoring['days_employed'] / 365
credit_scoring['years_employed'] = credit_scoring['days_employed'] / 365
credit_scoring.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,23.116912
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,11.026860
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,15.406637
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,11.300677
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,932.235814


Untuk sementara kita akan menghapus data yang memiliki nilai `years_employed` yang melebihi dari 49 tahun.

In [47]:
drop_year = credit_scoring.loc[(credit_scoring['years_employed'] > 49)]
credit_scoring['days_employed'] = credit_scoring['days_employed'].replace([drop_year['days_employed']], np.nan)
credit_scoring.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,23.116912
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,11.026860
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,15.406637
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,11.300677
4,0,NaN,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,932.235814


Dengan ini semua nilai yang bermasalah pada kolom `days_employed` telah diperbaiki. 

**Kolom `dob_years`**

Selanjutnya kita akan memeriksa distribusi pada kolom `dob_years`

In [48]:
# Memeriksa nilai yang mencurigakan pada kolom `dob_years`

print(sorted(credit_scoring['dob_years'].unique()))

[0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]


Terdapat *unique value* 0 pada kolom `dob_years`, ini merupakan nilai yang mencurigakan karena tidak mungkin ada nasabah yang berumur 0 tahun. Sebelum kita perbaiki nilainya mari hitung presentase nilai yang bermasalah terhadap keseluruhan *dataset*

In [49]:
# Menghitung presentase nilai yang bermasalah pada kolom `dob_years`

len(credit_scoring.loc[(credit_scoring['dob_years'] == 0)]) / len(credit_scoring) * 100

0.4692218350754936

Presentase data yang bermasalah cukup kecil yaitu 0.47% saja. Meski begitu mari perbaiki nilainya dengan nilai median ataupun nilai rata-rata dari keseluruhan nilai pada kolom tersebut.

In [50]:
# Mencari median dan nilai rata-rata

dob_years_avg = credit_scoring['dob_years'].mean()
dob_years_median = credit_scoring['dob_years'].median()

print(dob_years_avg)
print(dob_years_median)

43.29337979094077
42.0


Tidak ditemukan *outlier* yang signifikan sehingga kita akan menggunakan nilai rata-rata untuk mengisi nilai yang bermasalah.

In [51]:
# Mengatasi masalah pada kolom `dob_years`

credit_scoring['dob_years'] = credit_scoring['dob_years'].replace([0], dob_years_avg)

In [52]:
# Memeriksa hasilnya dan memastikan bahwa masalahnya telah diperbaiki

print(sorted(credit_scoring['dob_years'].unique()))

[19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 43.29337979094077, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0]


Nilai yang bermasalah pada kolom `dob_years` telah diperbaiki dangan cara menggantinya dengan nilai rata-rata. 

**Kolom `family_status`**

Kita akan melanjutkan pengecekan data pada kolom `family_status` dan melihat apakah ada data yang bermasalah.

In [53]:
# Mari kita lihat nilai untuk kolom ini

sorted(credit_scoring['family_status'].unique())

['civil partnership', 'divorced', 'married', 'unmarried', 'widow / widower']

Sepertinya kolom `family_status` tidak memiliki data yang bermasalah, sehingga kita bisa langsung berpindah pada kolom `gender`.

**Kolom `gender`**

In [54]:
# Melihat nilai dalam kolom `gender`

sorted(credit_scoring['gender'].unique())

['F', 'M', 'XNA']

Terdapat nilai yang mencurigakan yaitu `'XNA'`, mari hitung ada berapa data yang memiliki nila tersebut.

In [55]:
# Menghitung jumlah data dengan nilai 'XNA'

len(credit_scoring.loc[(credit_scoring['gender'] == 'XNA')])

1

Data yang memiliki nilai `'XNA'` ternyata hanya satu, sepertinya kita bisa mengabaikan datanya dengan cara menghapusnya.

In [56]:
# Menghapus data dengan nilai 'XNA'

drop_gender = credit_scoring[credit_scoring['gender']=='XNA'].index
credit_scoring = credit_scoring.drop(drop_gender).reset_index(drop=True)

sorted(credit_scoring['gender'].unique())

['F', 'M']

Data dengan nilai `'XNA'` telah dihapus.

**Kolom `income_type`**

 Selanjutnya kita beralih ke kolom `income_type` dan melihat apakah ada data yang bermasalah.

In [57]:
# Mari kita lihat nilai dalam kolom ini

sorted(credit_scoring['income_type'].unique())

['business',
 'civil servant',
 'employee',
 'entrepreneur',
 'paternity / maternity leave',
 'retiree',
 'student',
 'unemployed']

Sepertinya ada beberapa data yang memiliki nilai yang hampir mirip, misalnya `'entrepreneur'` bisa digabungkan dengan `'business'`. Mari kita gabungkan kedua data tersebut.

In [58]:
# Menggabungkan `'entrepreneur'` dengan `'business'`

credit_scoring['income_type'] = credit_scoring['income_type'].replace(['entrepreneur'], 'business')

In [59]:
sorted(credit_scoring['income_type'].unique())

['business',
 'civil servant',
 'employee',
 'paternity / maternity leave',
 'retiree',
 'student',
 'unemployed']

Data yang bermasalah telah diperbaiki. Selanjutnya mari kita periksa apakah ada duplikat dalam data kita.

### Menangani Duplikat <a id='duplicate'></a>

In [60]:
# Memeriksa duplikat

credit_scoring.duplicated().sum()

71

Terdapat sebanyak 71 data yang terduplikat, mari kita hapus duplikatnya.

In [61]:
# Mengatasi duplikat

credit_scoring = credit_scoring.drop_duplicates().reset_index(drop=True)

In [62]:
# Melakukan pemeriksaan terakhir untuk mengecek apakah dataKidit_scoring_clear.duplicated().sum()

credit_scoring.duplicated().sum()

0

In [63]:
# Memeriksa ukuran dataset yang baru

credit_scoring.shape

(21453, 13)

Kita telah memperbaiki data yang bermasalah dan telah menghapus sebanyak 71 data yang terduplikat sehingga menyisakan 18007 baris, dengan begitu kita bisa melanjutkan ke tahap selanjutnya.

## Tahap 4: Bekerja dengan nilai yang hilang <a id='missing_values'></a>

Setelah selesai mentransformasi data, selanjutnya mari perbaiki nilai yang hilang pada dataframe. Sebelumnya kita mulai memproses data yang hilang, kita akan memuat *dictionary* untuk beberapa nilai yang memiliki ID. Hal ini dilakukan dengan tujuan agar dapat mempermudah dalam mengamati datanya.

In [64]:
# Menemukan dictionary

dict_edu = credit_scoring[['education_id', 'education']].drop_duplicates().reset_index(drop=True)
dict_fam = credit_scoring[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True)

In [65]:
dict_edu

,education_id,education
0,0,bachelor's degree
1,1,secondary education
2,2,some college
3,3,primary education
4,4,graduate degree


In [66]:
dict_fam

,family_status_id,family_status
0,0,married
1,1,civil partnership
2,2,widow / widower
3,3,divorced
4,4,unmarried


### Memperbaiki nilai yang hilang di `total_income` <a id='missing_total_income'></a>

Kita telah mengetahui bahwa ada dua kolom yang memiliki nilai yang hilang, yaitu kolom `days_employed` dan kolom `total_income`. Kita akan mengatasi nilai yang hilang pada kolom `total_income` terlebih dahulu, untuk melakukannya kita harus membuat kategori usia nasabah terlebih dahulu.

In [67]:
# Mari kita tulis sebuah fungsi untuk menghitung kategori usia

def age_group(age):
    if age < 25:
        return 'youth'
    elif age < 35:
        return 'young adult'
    elif age < 50:
        return 'adult'
    elif age < 65:
        return 'elder'
    else: 
        return 'seniors' 

In [68]:
# Melakukan pengujian untuk melihat apakah fungsi yang telah dibuat dapat bekerja atau tidak

print(age_group(19))
print(age_group(33))
print(age_group(45))
print(age_group(60))
print(age_group(75))

youth
young adult
adult
elder
seniors


Fungsi bekerja dengan baik, mari kita buat kolom baru berdasarkan fungsi yang telah dibuat.

In [69]:
# Membuat kolom baru berdasarkan fungsi

credit_scoring['age_group'] = credit_scoring['dob_years'].apply(age_group)

In [70]:
# Memeriksa nilai di dalam kolom baru

credit_scoring.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,age_group
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,23.116912,adult
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase,11.026860,adult
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,15.406637,young adult
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,11.300677,young adult
4,0,NaN,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,932.235814,elder
5,0,926.185831,27.0,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,2.537495,young adult
6,0,2879.202052,43.0,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,7.888225,adult
7,0,152.779569,50.0,secondary education,1,married,0,M,employee,0,21731.829,education,0.418574,elder
8,2,6929.865299,35.0,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,18.985932,adult
9,0,2188.756445,41.0,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,5.996593,adult


In [71]:
# Membuat tabel tanpa nilai yang hilang dan menampilkan beberapa barisnya untuk memastikan semuanya berjalan dengan baik

credit_scoring_null = credit_scoring.dropna()
credit_scoring_null.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,age_group
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,23.116912,adult
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase,11.026860,adult
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,15.406637,young adult
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,11.300677,young adult
5,0,926.185831,27.0,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,2.537495,young adult
6,0,2879.202052,43.0,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,7.888225,adult
7,0,152.779569,50.0,secondary education,1,married,0,M,employee,0,21731.829,education,0.418574,elder
8,2,6929.865299,35.0,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,18.985932,adult
9,0,2188.756445,41.0,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,5.996593,adult
10,2,4171.483647,36.0,bachelor's degree,0,married,0,M,business,0,18230.959,buy real estate,11.428722,adult


Kita akan mencari nilai rata-rata dan median dari kolom `total_income` berdasarkan kolom `income_type`.

In [72]:
# Nilai rata-rata untuk pendapatan berdasarkan faktor `income_type`

total_income_avg = credit_scoring_null.groupby('income_type')['total_income'].mean().reset_index()
total_income_avg

,income_type,total_income
0,business,32397.115286
1,civil servant,27343.729582
2,employee,25820.445420
3,paternity / maternity leave,8612.661000
4,student,15712.260000


In [73]:
# Nilai median untuk pendapatan berdasarkan faktor `income_type`

total_income_median = credit_scoring_null.groupby('income_type')['total_income'].median().reset_index()
total_income_median

,income_type,total_income
0,business,27577.2720
1,civil servant,24071.6695
2,employee,22814.0140
3,paternity / maternity leave,8612.6610
4,student,15712.2600


Nilai rata-rata memiliki outlier yang signifikan sehingga kita akan menggunakan nilai median sebagai nilai yang akan menggantikan nilai yang bermasalah.

In [74]:
# Menulis fungsi yang akan digunakan untuk mengisi nilai yang hilang

def fill_missing_income (dataframe, agg_column, value_column):
    grouped_values = dataframe.groupby(agg_column)[value_column].median().reset_index()
    size = len(grouped_values)
    for i in range(size):
        group = grouped_values[agg_column][i]
        value = grouped_values[value_column][i]
        dataframe.loc[(dataframe[agg_column]==group) & (dataframe[value_column].isna()), value_column] = value
    return dataframe

In [75]:
# Memeriksa bagaimana nilai di dalam kolom baru

grouped_values = credit_scoring.groupby('income_type')['total_income'].median().reset_index()
size = len(grouped_values)
for i in range(size):
    group = grouped_values['income_type'][i]
    value = grouped_values['total_income'][i]
    credit_scoring.loc[(credit_scoring['income_type']==group) & (credit_scoring['total_income'].isna()), 'total_income'] = value
    print(group, value)

business 27577.272
civil servant 24071.6695
employee 22815.103499999997
paternity / maternity leave 8612.661
retiree 18962.318
student 15712.26
unemployed 21014.360500000003


Fungsi berjalan dengan baik, mari terapkan fungsi tersebut ke semua baris.

In [76]:
# Terapkan fungsi tersebut ke setiap baris
fill_missing_income (credit_scoring, 'income_type', 'total_income').head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,age_group
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,23.116912,adult
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase,11.026860,adult
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,15.406637,young adult
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,11.300677,young adult
4,0,NaN,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,932.235814,elder


In [77]:
# Memeriksa apakah kita mendapatkan kesalahan
credit_scoring['total_income'].isna().sum() 

0

Jumlah nilai yang hilang pada kolom `total_income` adalah 0, artinya semua nilai yang hilang telah diperbaiki.

Mari kita hitung jumlah nilainya apakah sudah sesuai dengan jumlah nilai di kolom lain.

In [78]:
# Periksa jumlah entri di kolom

credit_scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21453 non-null  int64  
 1   days_employed     15904 non-null  float64
 2   dob_years         21453 non-null  float64
 3   education         21453 non-null  object 
 4   education_id      21453 non-null  int64  
 5   family_status     21453 non-null  object 
 6   family_status_id  21453 non-null  int64  
 7   gender            21453 non-null  object 
 8   income_type       21453 non-null  object 
 9   debt              21453 non-null  int64  
 10  total_income      21453 non-null  float64
 11  purpose           21453 non-null  object 
 12  years_employed    19350 non-null  float64
 13  age_group         21453 non-null  object 
dtypes: float64(4), int64(4), object(6)
memory usage: 2.3+ MB


Dengan ini jumlah nilai pada kolom `total_income` sudah sama dengan jumlah nilai di kolom lain.

###  Memperbaiki nilai di `days_employed` <a id='missing_days_employed'></a>

Dalam memperbaiki nilai yang hilang pada kolom `days_employed` kita akan menggunakan nilai rata-rata dan nilai median dari kolom tersebut berdasarkan kolom `age_group` yang telah dibuat sebelumnya.

In [79]:
# Distribusi rata-rata dari `days_employed` berdasarkan parameter `age_group`

days_employed_avg = credit_scoring_null.groupby('age_group')['days_employed'].mean().reset_index()
days_employed_avg

,age_group,days_employed
0,adult,2560.209423
1,elder,3322.724096
2,seniors,4010.478357
3,young adult,1586.946577
4,youth,852.074411


In [80]:
# Distribusi median dari `days_employed` berdasarkan parameter `age_group`

days_employed_median = credit_scoring_null.groupby('age_group')['days_employed'].median().reset_index()
days_employed_median

,age_group,days_employed
0,adult,1919.675289
1,elder,2304.455212
2,seniors,2876.221697
3,young adult,1284.275365
4,youth,744.016267


Untuk menangani data pada kolom `days_employed` kita akan meggunankan nilai rata-rata dikarenakan nilainya lebih masuk akal dibandingkan nilai median yang nilainya terhitung kecil untuk jumlah hari bekerja.

In [81]:
# Membuat fungsi yang menghitung median berdasarkan parameter `age_group`

def fill_missing_days (dataframe, agg_column, value_column):
    grouped_values = dataframe.groupby(agg_column)[value_column].mean().reset_index()
    size = len(grouped_values)
    for i in range(size):
        group = grouped_values[agg_column][i]
        value = grouped_values[value_column][i]
        dataframe.loc[(dataframe[agg_column]==group) & (dataframe[value_column].isna()), value_column] = value
    return dataframe

In [82]:
# Memeriksa apakah fungsi tersebut dapat bekerja

grouped_values = credit_scoring.groupby('age_group')['days_employed'].mean().reset_index()
size = len(grouped_values)
for i in range(size):
    group = grouped_values['age_group'][i]
    value = grouped_values['days_employed'][i]
    credit_scoring.loc[(credit_scoring['age_group']==group) & (credit_scoring['days_employed'].isna()), 'days_employed'] = value
    print(group, value)

adult 2560.2094230728835
elder 3322.7240958353273
seniors 4010.47835666443
young adult 1586.9465771465298
youth 852.074410928637


In [83]:
# Terapkan fungsi ke income_type

fill_missing_days (credit_scoring, 'age_group', 'days_employed').head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,age_group
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,23.116912,adult
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase,11.026860,adult
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,15.406637,young adult
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,11.300677,young adult
4,0,3322.724096,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,932.235814,elder


Sebelum memeriksa nilai yang hilang mari perbaiki kolom `years_employed` terlebih dahulu.

In [84]:
# Membuat kolom `years_employed`

credit_scoring['days_employed'] / 365
credit_scoring['years_employed'] = credit_scoring['days_employed'] / 365
credit_scoring.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,age_group
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,23.116912,adult
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase,11.026860,adult
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,15.406637,young adult
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,11.300677,young adult
4,0,3322.724096,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,9.103354,elder


In [85]:
# Memeriksa apakah masih ada nilai yang hilang

credit_scoring['days_employed'].isna().sum()

0

Kolom `total_income` memiliki jumlah nilai yang hilang sebanyak 0, artinya semua nilai yang hilang telah diperbaiki.

In [86]:
# Periksa entri di semua kolom - pastikan kita memperbaiki semua nilai yang hilang

credit_scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21453 non-null  int64  
 1   days_employed     21453 non-null  float64
 2   dob_years         21453 non-null  float64
 3   education         21453 non-null  object 
 4   education_id      21453 non-null  int64  
 5   family_status     21453 non-null  object 
 6   family_status_id  21453 non-null  int64  
 7   gender            21453 non-null  object 
 8   income_type       21453 non-null  object 
 9   debt              21453 non-null  int64  
 10  total_income      21453 non-null  float64
 11  purpose           21453 non-null  object 
 12  years_employed    21453 non-null  float64
 13  age_group         21453 non-null  object 
dtypes: float64(4), int64(4), object(6)
memory usage: 2.3+ MB


Semua nilai yang bermasalah dalam dataset telah diperbaiki.

## Tahap 5: Pengkategorian Data <a id='categorize'></a>

Data yang bermasalah telah diperbaiki, begitu pula dengan nilai yang hilang. Selanjutnya mari melakukan pengkategorian data beberapa kolom dalam keseluruhan dataset.

Pertama-tama kita kelompokan data pada kolom `purpose`.

In [87]:
# Menampilkan kolom `purpose`

credit_scoring[['purpose']]

,purpose
0,purchase of the house
1,car purchase
2,purchase of the house
3,supplementary education
4,to have a wedding
...,...
21448,housing transactions
21449,purchase of a car
21450,property
21451,buying my own car


Memeriksa nilai unik pada kolom `purpose`.

In [88]:
# Memeriksa nilai unik

sorted(credit_scoring['purpose'].unique())

['building a property',
 'building a real estate',
 'buy commercial real estate',
 'buy real estate',
 'buy residential real estate',
 'buying a second-hand car',
 'buying my own car',
 'buying property for renting out',
 'car',
 'car purchase',
 'cars',
 'construction of own property',
 'education',
 'getting an education',
 'getting higher education',
 'going to university',
 'having a wedding',
 'housing',
 'housing renovation',
 'housing transactions',
 'profile education',
 'property',
 'purchase of a car',
 'purchase of my own house',
 'purchase of the house',
 'purchase of the house for my family',
 'real estate transactions',
 'second-hand car purchase',
 'supplementary education',
 'to become educated',
 'to buy a car',
 'to get a supplementary education',
 'to have a wedding',
 'to own a car',
 'transactions with commercial real estate',
 'transactions with my real estate',
 'university education',
 'wedding ceremony']

Kita akan mengategorikan datanya menjadi hanya 5 nilai unik yaitu `'car'`, `'real estate'`, `'wedding'`, `'education'`, dan `'unidentified'`.

In [89]:
# Mari kita tulis sebuah fungsi untuk mengategorikan data berdasarkan topik umum

def categorize_purpose(row):
    if 'car' in row:
        return 'car'
    elif 'hous' in row or 'prop' in row or 'real est' in row:
        return 'real estate'
    elif 'wedd' in row:
        return 'wedding'
    elif 'educ' in row or 'uni' in row:
        return 'education'
    else:
        return 'unidentified'

In [90]:
# Membuat kolom yang memuat kategori

credit_scoring['categorize_purpose'] = credit_scoring['purpose'].apply(categorize_purpose)
credit_scoring.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,age_group,categorize_purpose
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,23.116912,adult,real estate
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase,11.026860,adult,car
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,15.406637,young adult,real estate
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,11.300677,young adult,education
4,0,3322.724096,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,9.103354,elder,wedding


In [91]:
# Menghitung nilai setiap kategori

credit_scoring['categorize_purpose'].value_counts()

real estate    10810
car             4306
education       4013
wedding         2324
Name: categorize_purpose, dtype: int64

Kolom `purpose` telah dikategorikan dan kita telah menghitung masing-masing nilainya.

Selanjutnya mari kelompokan data pada kolom `total_income` yang berisi data numerik.

In [92]:
# Melihat semua data numerik di kolom yang Anda pilih untuk pengkategorian

credit_scoring[['total_income']]

,total_income
0,40620.102
1,17932.802
2,23341.752
3,42820.568
4,25378.572
...,...
21448,35966.698
21449,24959.969
21450,14347.610
21451,39054.888


Kita akan membagi tingkat pendapatan menjadi 5 kategori yaitu `'small'`, `'avarage'`, `'above avarege'`, `'high'`, dan `'very high'`.

In [93]:
# Membuat fungsi yang melakukan pengkategorian menjadi kelompok numerik yang berbeda berdasarkan rentang

def income_level(income):
    if (income > 0) and (income <= 10000):
        return 'small'
    if (income > 10000) and (income <= 20000):
        return 'avarage'
    if (income > 20000) and (income <= 25000):
        return 'above avarege'
    if (income > 25000) and (income <= 50000):
        return 'high'
    if income > 50000:
        return 'very high'

In [94]:
# Membuat kolom yang memuat kategori

credit_scoring['income_level'] = credit_scoring['total_income'].apply(income_level)
credit_scoring.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,age_group,categorize_purpose,income_level
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,23.116912,adult,real estate,high
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase,11.026860,adult,car,avarage
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,15.406637,young adult,real estate,above avarege
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,11.300677,young adult,education,high
4,0,3322.724096,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,9.103354,elder,wedding,high


In [95]:
# Menghitung setiap nilai kategori untuk melihat pendistribusiannya

credit_scoring['income_level'].value_counts()

high             7784
avarage          6829
above avarege    4594
very high        1320
small             926
Name: income_level, dtype: int64

In [96]:
credit_scoring['income_level'].unique()

array(['high', 'avarage', 'above avarege', 'small', 'very high'],
      dtype=object)

## Tahap 6: Memeriksa hipotesis <a id='hypotheses'></a>

### Hipotesis 1 <a id='hypotheses_1'></a>

**Apakah terdapat korelasi antara memiliki anak dengan melakukan pelunasan tepat waktu?**

In [97]:
# Memeriksa data anak dan data pelunasan tepat waktu
credit_scoring[['children', 'debt']].head()

,children,debt
0,1,0
1,1,0
2,0,0
3,3,0
4,0,0


In [98]:
sorted(credit_scoring['children'].unique())

[0, 1, 2, 3, 4, 5]

In [99]:
# Menghitung gagal bayar berdasarkan jumlah anak
credit_scoring[['children']].loc[(credit_scoring['debt'] == 1)].value_counts()

children
0           1063
1            445
2            202
3             27
4              4
dtype: int64

In [100]:
credit_scoring.loc[(credit_scoring['debt']==1) & (credit_scoring['children']==5)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,age_group,categorize_purpose,income_level


**Kesimpulan**

Hasil analisa menunjukan fakta yang cukup unik, dimana semakin banyak anak yang dimiliki justru semakin sedikit kemungkinan nasabah melakukan gagal bayar. Hal ini dibuktikan dengan 1063 nasabah yang gagal bayar dan ternyata tidak memiliki anak, namun hanya ada 4 nasabah dengan 4 anak yang melakukan gagal bayar dan tidak ada yg melakukan gagal bayar bagi nasabah yang memiliki 5 anak.

### Hipotesis 2 <a id='hypotheses_2'></a>

**Apakah terdapat korelasi antara status keluarga dengan pelunasan tepat waktu?**

In [101]:
# Memeriksa data status keluarga dan pelunasan tepat waktu

credit_scoring[['family_status', 'debt']].head()

,family_status,debt
0,married,0
1,married,0
2,married,0
3,married,0
4,civil partnership,0


In [102]:
sorted(credit_scoring['family_status'].unique())

['civil partnership', 'divorced', 'married', 'unmarried', 'widow / widower']

In [103]:
# Menghitung gagal bayar berdasarkan status keluarga
credit_scoring[['family_status']].loc[(credit_scoring['debt'] == 1)].value_counts()

family_status    
married              931
civil partnership    388
unmarried            274
divorced              85
widow / widower       63
dtype: int64

**Kesimpulan**

Nasabah yang telah menikah memiliki jumlah gagal bayar paling tinggi sedangkan nasabah yang telah bercerai dan telah menjadi janda/duda memiliki jumlah gagal bayar yang jauh lebih rendah.

### Hipotesis 3 <a id='hypotheses_3'></a>

**Apakah terdapat korelasi antara tingkat pendapatan dengan membayar kembali tepat waktu?**

In [104]:
# Memeriksa data tingkat pendapatan dan pelunasan tepat waktu

credit_scoring[['income_level', 'debt']].head()

,income_level,debt
0,high,0
1,avarage,0
2,above avarege,0
3,high,0
4,high,0


In [105]:
sorted(credit_scoring['income_level'].unique())

['above avarege', 'avarage', 'high', 'small', 'very high']

In [106]:
# Menghitung gagal bayar berdasarkan tingkat pendapatan
credit_scoring[['income_level']].loc[(credit_scoring['debt'] == 1)].value_counts()

income_level 
high             613
avarage          585
above avarege    393
very high         92
small             58
dtype: int64

**Kesimpulan**

Uniknya semakin rendah tingkat pendapatan seorang nasabah, semakin rendah pula kemungkinan gagal bayarnya.

### Hipotesis 4 <a id='hypotheses_4'></a>

**Bagaimana tujuan kredit memengaruhi tingkat gagal bayar?**

In [107]:
sorted(credit_scoring['categorize_purpose'].unique())

['car', 'education', 'real estate', 'wedding']

In [108]:
# Memeriksa persentase tingkat gagal bayar untuk setiap tujuan kredit

credit_scoring[['categorize_purpose']].loc[(credit_scoring['debt'] == 1)].value_counts() / len(credit_scoring) * 100

categorize_purpose
real estate           3.645178
car                   1.878525
education             1.724701
wedding               0.867012
dtype: float64

**Kesimpulan**

`'real estate'` menjadi tujuan kredit dengan presentase gagal bayar yang paling tinggi diantara tujuan kredit lainnya yaitu sebesar 3.65%, disusul dengan `'car'` sebesar 1.88%, `'education'` sebesar 1.72%, dan yang terakhir adalah `'wedding'` dengan hanya sebesar 0.87%.

## Kesimpulan <a id='end'></a>

Kita telah mentransformasi semua data yang memiliki nilai yang bermasalah pada setiap kolom, kemudian kita juga telah menghapus sejumlah data yang terduplikat dan berhasil mengelompokan nasabah berdasarkan umurnya yang kemudian kita gunakan untuk membantu kita dalam mencari nilai median dari kolom `days_employed` berdasarkan kelompok umur tersebut. Nilai median tadi kemudian kita gunakan sebagai nilai yang akan mengisi nilai yang hilang pada kolom `days_employed`. Kita juga telah mengganti nilai yang hilang pada kolom `total_income` dengan nilai rata-rata pada kolom tersebut berdasarkan kolom `income_type`. Selanjutnya kita telah mengelompokan data pada kolom `purpose` dan `total_income` menjadi `categorize_purpose` dan `income_level`.

Kita telah menguji empat hipotesis berikut:
1.	Apakah terdapat korelasi antara memiliki anak dengan melakukan pelunasan tepat waktu?
2.	Apakah terdapat korelasi antara status keluarga dengan pelunasan tepat waktu?
3.	Apakah terdapat korelasi antara tingkat pendapatan dengan membayar kembali tepat waktu?
4.	Bagaimana tujuan kredit memengaruhi tingkat gagal bayar?

Setelah menganalisis data, kita dapat menyimpulkan:

1.	Semakin banyak anak yang dimiliki justru semakin sedikit kemungkinan nasabah melakukan gagal bayar.

Hipotesis pertama dapat diterima sepenuhnya.

2.	Nasabah yang telah menikah dan masih menjalani pernikahannya cenderung memiliki presentase gagal bayar yang cukup tinggi dibandingkan nasabah yang telah bercerai maupun yang telah menjadi janda/duda.

Hipotesis kedua juga dapat diterima.

3.	Semakin rendah tingkat pendapatan seorang nasabah, semakin rendah pula kemungkinan gagal bayarnya.

Ini merupakan fakta yang cukup unik, dan hipotesa ketiga juga dapat kita terima.

4.	'real estate' menjadi tujuan kredit dengan presentase gagal bayar yang paling tinggi diantara tujuan kredit lainnya yaitu sebesar 3.65%, disusul dengan 'car' sebesar 1.88%, 'education' sebesar 1.72%, dan yang terakhir adalah 'wedding' dengan hanya sebesar 0.87%.

Real estate menjadi tujuan kredit paling favorit namun memiliki presentase gagal bayar yang cukup tinggi juga. Hal ini membuktikan bahwa hipotesa keempat juga benar.

**Hasil analisa membuktikan bahwa semua hipotesa terbukti dan dapat diterima.**